<a href="https://colab.research.google.com/github/Yussufkadir/Neural-Network-SMS-Text-Classifier/blob/main/Copy_of_fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers, models
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer

print(tf.__version__)

2.20.0-dev20250221


In [33]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2025-02-22 15:49:13--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv.2’

train-data.tsv.2    100%[===================>] 349.84K  --.-KB/s    in 0.05s   

2025-02-22 15:49:14 (6.32 MB/s) - ‘train-data.tsv.2’ saved [358233/358233]

--2025-02-22 15:49:14--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv.2’

valid-data.tsv.2    100%[========

In [34]:
train_df = pd.read_csv("train-data.tsv", sep='\t')
test_df = pd.read_csv("valid-data.tsv", sep='\t')

In [35]:
train_df = train_df.rename(columns={'ahhhh...just woken up!had a bad dream about u tho,so i dont like u right now :) i didnt know anything about comedy night but i guess im up for it.': 'messages'})

In [36]:
test_df = test_df.rename(columns={'i am in hospital da. . i will return home in evening': 'messages'})

In [51]:
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(train_df['messages'])
X_test = vectorizer.transform(test_df['messages'])

X_train = X_train.toarray()
X_test = X_test.toarray()

train_labels = np.array([1 if label == 'ham' else 0 for label in train_df['ham']], dtype=np.int32)
test_labels = np.array([1 if label == 'ham' else 0 for label in test_df['ham']], dtype=np.int32)


model = models.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, train_labels, epochs=10, batch_size=32, validation_data=(X_test, test_labels))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.8759 - loss: 0.4202 - val_accuracy: 0.9878 - val_loss: 0.0617
Epoch 2/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.9942 - loss: 0.0282 - val_accuracy: 0.9849 - val_loss: 0.0528
Epoch 3/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9982 - loss: 0.0098 - val_accuracy: 0.9892 - val_loss: 0.0434
Epoch 4/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.9995 - loss: 0.0030 - val_accuracy: 0.9892 - val_loss: 0.0477
Epoch 5/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9997 - loss: 0.0018 - val_accuracy: 0.9892 - val_loss: 0.0541
Epoch 6/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9999 - loss: 8.0418e-04 - val_accuracy: 0.9892 - val_loss: 0.0578
Epoch 7/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 1.0000 - loss: 5.3777e-04 - val_accuracy: 0.9892 - val_loss: 0.0603
Epoch 8/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 1.0000 - loss: 3.8332e-04 - v

In [54]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  message_vectorized = vectorizer.transform([pred_text])

  prediction = model.predict(message_vectorized)

  if prediction > 0.5:
    prediction = [prediction, 'ham']
  else:
    prediction = [prediction, 'spam']

  return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
[array([[0.9998677]], dtype=float32), 'ham']


In [55]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
You passed the challenge. Great job!
